In [93]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
from scipy.spatial import cKDTree
import numpy as np
import requests
import json
from tqdm.auto import tqdm
tqdm.pandas()  # "tqdm>=4.9.0"
school_path = "../data/raw/school_zones/school_zones.csv"
ptv_path = "../data/raw/ptv_zones/ptv_zones.csv"

/home/even/repo/project-2-group-real-estate-industry-project-2/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Rental

In [8]:
path = "../data/raw/oldlistings_rent_2.json"
property_df = pd.read_json(open(path, "r"))

In [9]:
property_df

,lat,lng,rented_prices,address,meta_data
0,-37.813730,144.955580,"[{'date': 'August 2023', 'price': '$800 per we...","201/560 LONSDALE STREET, MELBOURNE","[{'label': 'bed', 'description': 'Bed', 'quant..."
1,-37.813730,144.955580,"[{'date': 'August 2023', 'price': '$720 per we...","1702/560 LONSDALE STREET, MELBOURNE","[{'label': 'bed', 'description': 'Bed', 'quant..."
2,-37.810280,144.956670,"[{'date': 'August 2023', 'price': '$830 per we...","2007/228 ABECKETT STREET, MELBOURNE","[{'label': 'bed', 'description': 'Bed', 'quant..."
3,-37.813590,144.955720,"[{'date': 'August 2023', 'price': '$660 per we...","901/21 HEALEYS LANE, MELBOURNE","[{'label': 'bed', 'description': 'Bed', 'quant..."
4,-37.813700,144.953930,"[{'date': 'August 2023', 'price': '$830 per we...","3801/601 LITTLE LONSDALE STREET, MELBOURNE","[{'label': 'bed', 'description': 'Bed', 'quant..."
...,...,...,...,...,...
539050,-37.840959,145.032227,"[{'date': 'November 2007', 'price': '$575 pw'}...","3/422-426 GLENFERRIE ROAD, KOOYONG","[{'label': 'bed', 'description': 'Bed', 'quant..."
539051,-37.842220,145.033818,"[{'date': 'October 2007', 'price': '$1,700 pw'...","9 MERNDA ROAD, KOOYONG","[{'label': 'bed', 'description': 'Bed', 'quant..."
539052,-37.841322,145.030886,"[{'date': 'February 2007', 'price': '$575 pw'}...","27 POWER AVENUE, KOOYONG","[{'label': 'bed', 'description': 'Bed', 'quant..."
539053,-37.843799,145.032879,"[{'date': 'February 2007', 'price': '$350 pw'}...","699A TOORAK ROAD, KOOYONG","[{'label': 'bed', 'description': 'Bed', 'quant..."


In [10]:
property_df['geometry'] = property_df.apply(lambda row: Point(row['lng'], row['lat']), axis=1)

In [7]:
school  = pd.read_csv(school_path)
school['geometry'] = school['geometry'].apply(wkt.loads)
gdf_school = gpd.GeoDataFrame(school, geometry='geometry')

In [8]:
gdf_school

,School_Name,geometry
0,Lockwood Primary School,"POLYGON ((144.19743 -36.82513, 144.20366 -36.8..."
1,Lockwood South Primary School,"POLYGON ((143.99198 -36.80865, 143.99179 -36.8..."
2,Heathcote Primary School,"POLYGON ((144.6649 -37.02629, 144.88917 -36.91..."
3,Redesdale Mia Mia Primary School,"POLYGON ((144.55801 -36.89947, 144.51209 -36.9..."
4,Axedale Primary School,"POLYGON ((144.71365 -36.74947, 144.66726 -36.7..."
...,...,...
1582,Matthew Flinders Girls Secondary College,"POLYGON ((144.4229 -38.08573, 144.41875 -38.08..."
1583,Canterbury Girls Secondary College,"POLYGON ((145.03764 -37.80765, 145.09942 -37.8..."
1584,Mentone Girls Secondary College,"POLYGON ((145.07547 -38.00074, 145.0713 -37.99..."
1585,Melbourne Girls College,"POLYGON ((145.02534 -37.79516, 145.04552 -37.8..."


In [40]:
gdf_houses_with_zones = gpd.sjoin(gpd.GeoDataFrame(df), gdf_school, how="inner", predicate="within")

In [43]:
# 1127421
in_zone_set = set(gdf_houses_with_zones.reset_index()['index'].values)

In [47]:
df.iloc[1].name

1

In [48]:
# gdf_houses_with_zones.reset_index()
df["inschool"] = df.reset_index().apply(lambda row: row.name in in_zone_set, axis=1)

In [52]:
df["inschool"].sum() / df.shape[0]

np.float64(0.9896411312389274)

In [ ]:
df

# PTV

In [34]:
paths = ["../data/raw/PTV/2 - Metropolitan Train/stops_2_sa2.csv" ,"../data/raw/PTV/3 - Metropolitan Tram/stops_3_sa2.csv", "../data/raw/PTV/4 - Metropolitan Bus/stops_4_sa2.csv" ]
total_ptv_coords = []
for path in paths:
    ptv = pd.read_csv(path)
    total_ptv_coords = total_ptv_coords + [ptv[['stop_lon','stop_lat']].values]
    # total_ptv_coords = total_ptv_coords + ptv[['stop_lon','stop_lat']].values\
total_ptv_coords = np.concatenate(total_ptv_coords)

In [7]:
ptv_df = pd.read_csv(paths[0])
ptv_df["geometry"] = ptv_df.apply(lambda row: Point(row['stop_lon'], row['stop_lat']), axis=1)
ptv_gdf = gpd.GeoDataFrame(ptv_df, geometry='geometry')

In [20]:
property_coords = property_df[['lng', 'lat']].values

In [17]:
ptv_stop_coords = ptv_gdf[['stop_lon','stop_lat']].values

In [37]:
tree = cKDTree(total_ptv_coords)
distances, indices = tree.query(property_coords, k=3)

In [41]:
property_df["near_ptv_stop_1"] = indices[:, 0]
property_df["near_ptv_stop_2"] = indices[:, 1]
property_df["near_ptv_stop_3"] = indices[:, 2]

In [42]:
property_df

,lat,lng,rented_prices,address,meta_data,geometry,near_ptv_stop_1,near_ptv_stop_2,near_ptv_stop_3
0,-37.813730,144.955580,"[{'date': 'August 2023', 'price': '$800 per we...","201/560 LONSDALE STREET, MELBOURNE","[{'label': 'bed', 'description': 'Bed', 'quant...",POINT (144.95558001 -37.81373001),16,517,2142
1,-37.813730,144.955580,"[{'date': 'August 2023', 'price': '$720 per we...","1702/560 LONSDALE STREET, MELBOURNE","[{'label': 'bed', 'description': 'Bed', 'quant...",POINT (144.95558001 -37.81373001),16,517,2142
2,-37.810280,144.956670,"[{'date': 'August 2023', 'price': '$830 per we...","2007/228 ABECKETT STREET, MELBOURNE","[{'label': 'bed', 'description': 'Bed', 'quant...",POINT (144.95667001 -37.81028001),1747,3372,434
3,-37.813590,144.955720,"[{'date': 'August 2023', 'price': '$660 per we...","901/21 HEALEYS LANE, MELBOURNE","[{'label': 'bed', 'description': 'Bed', 'quant...",POINT (144.95572001 -37.81359001),517,2142,16
4,-37.813700,144.953930,"[{'date': 'August 2023', 'price': '$830 per we...","3801/601 LITTLE LONSDALE STREET, MELBOURNE","[{'label': 'bed', 'description': 'Bed', 'quant...",POINT (144.95393001 -37.8137001),1746,3371,3370
...,...,...,...,...,...,...,...,...,...
539050,-37.840959,145.032227,"[{'date': 'November 2007', 'price': '$575 pw'}...","3/422-426 GLENFERRIE ROAD, KOOYONG","[{'label': 'bed', 'description': 'Bed', 'quant...",POINT (145.032227 -37.840959),1422,3047,3048
539051,-37.842220,145.033818,"[{'date': 'October 2007', 'price': '$1,700 pw'...","9 MERNDA ROAD, KOOYONG","[{'label': 'bed', 'description': 'Bed', 'quant...",POINT (145.033818 -37.84222),1422,3047,3046
539052,-37.841322,145.030886,"[{'date': 'February 2007', 'price': '$575 pw'}...","27 POWER AVENUE, KOOYONG","[{'label': 'bed', 'description': 'Bed', 'quant...",POINT (145.030886 -37.841322),1422,3047,3046
539053,-37.843799,145.032879,"[{'date': 'February 2007', 'price': '$350 pw'}...","699A TOORAK ROAD, KOOYONG","[{'label': 'bed', 'description': 'Bed', 'quant...",POINT (145.032879 -37.843799),3045,1420,1419


In [89]:
def get_time_proximity(coordinates):
    url = "http://localhost:8080/ors/v2/directions/foot-walking"
    body = {"coordinates": coordinates}
    # print(body)
    response = requests.post(url, json=body)

    if response.status_code == 200:
        return response.json()["routes"][0]["summary"]["duration"]
    else:
        if(response.json()["error"]["code"] == 2004):
            # distance is too long (> 100000.0 m)
            return -1
        if(response.json()["error"]["code"] == 2010):
            # "error":{"code":2010,"message":"Could not find routable point within a radius of 400.0 meters of specified coordinate
            return -1
        print(body)
        print(response.text)
        raise Exception("Error in request")

# # Example usage
# directions = get_time_proximity([[144.96332, -37.8140], [144.96332, -37.8120]])
# print(directions)

In [90]:
def get_time_proximity_from_property(row, ptv_stop_coords,i):
    coordinates = [[row['lng'], row['lat']], ptv_stop_coords[row[f"near_ptv_stop_{i}"]].tolist()]
    return get_time_proximity(coordinates)

In [94]:
property_df["time_to_ptv_1"] = property_df.progress_apply(lambda row: get_time_proximity_from_property(row, total_ptv_coords, 1), axis=1)

  8%|▊         | 42441/539055 [04:43<55:18, 149.64it/s]  


KeyError: 'duration'

In [44]:
str([[144.96332, -37.8140], [144.96332, -37.8120]])

'[[144.96332, -37.814], [144.96332, -37.812]]'